In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import Markdown, display, clear_output
import matplotlib.pyplot as plt

DAYS_3_WEEKS = 21 # how many days are in 3 weeks 
DAYS_6_MONTHS = 182 # how many days are in 6 months

## Introduction

**The primary objective of this EDA project is to provide restaurent business owners with insight on their performance as measured by user reviews.** 

**Specifically, we would like to alert restaurents about periods in which the rating average deviates greatly from the rating of the 6 months period before, for good or for bad, so they could pay extra attention to the reviews from this time period.**

**By exaimaning the data from each outlier period, the business can gain insight on some practices they should maintain, and practices they could improve on.
The aim is also to alert business owners in semi real-time (3-week delay), so they could make the adjustments as soon as possible, and perhaps prevent a negative trend in reviews and customer satisfication in general from starting.**

**Read the files**

In [ ]:
# read the csv files
df_1 = pd.read_csv('/kaggle/input/precovid_reviews.csv')
df_2 = pd.read_csv('/kaggle/input/postcovid_reviews.csv')

## Data Observation and Cleaning

**See the date range for each dataset. Is it reasonable?**

In [ ]:
df_1['date_'] = pd.to_datetime(df_1['date_'])
df_2['date_'] = pd.to_datetime(df_2['date_'])
print('Date Range Pre-Covid:', df_1['date_'].dt.date.min(), '-', df_1['date_'].dt.date.max())
print('Date Range Post-Covid:', df_2['date_'].dt.date.min(), '-', df_2['date_'].dt.date.max())

**It seems very reasonable. Yelp was founded in 2004, and covid started around the begining of 2020**

**Also, we can see from that that there's no overlap between the 2 datasets**

**Merge the 2 datasets, to achieve full view of all reviews**

In [ ]:
# merge the 2 dataframes. as seen before, there's no overlap between them, so we can use concat
df = pd.concat([df_1,df_2])

**Take a peek at the data, to get a grasp of it**

In [ ]:
df.head()

**Display basic info about the data set**

In [ ]:
df.info()

**Are there any missing values?**

In [ ]:
df.isna().sum()

**Looks like there aren't any missing values in coloums that are crucial to our analysis - thats's good.**

**We'll mess with address a little bit later**

**Is every row a distinct review, i.e does each row have a distinct review id, or could the data be corrupted?**


In [ ]:
print('Is every row assigned a distinct review id?', not bool(df['review_id'].duplicated().sum()))

**Do we have duplicate reviews (same text), even though each one has a unique id?**

In [ ]:
# checks to see if we have duplicate reviews with length >= 30, as reviews with this length with the same text
# are highly unlikely to *not* be duplicates
duplicate_texts = df[df['text_'].str.len() >= 30]
duplicate_texts = duplicate_texts[duplicate_texts.duplicated(subset='text_', keep=False)]
print('Number of duplicate reviews:', duplicate_texts['text_'].nunique())

**Show the duplicate reviews which appear the most**

In [ ]:
duplicate_texts['text_'].value_counts().sort_values(ascending=False).head(20)

**By looking at the snippet of reviews, seems like besides: 'Your review helps others learn about great local businesses. Please don't review this business if you received a freebie for writing this review, or if you're connected in any way to the owner or employees.', which could be the default review text at a specific time which users didn't change, all the others are indeed duplicate reviews. Maybe users left the same review on diffrent branches of the same chain, maybe they are planted reviews by the restaurents, an accidental double posting from the user, or even a yelp bug. We won't investigate it further - we'll just drop all occourences but the first one. Not a perfect solution - but will do**

**For each duplicate review, drop all but the first one. Only for reviews of which length is 30 or more, as they are highly unlikely *not* to be duplicates**

In [ ]:
mask = df['text_'].str.len() >= 30
df.loc[mask] = df[mask].drop_duplicates(subset='text_', keep='first').reset_index(drop=True)

**Is there only one name assigned to each business_id? or could the data be corrupted?**

In [ ]:
print('Is there a distinct name for each business_id:', df.groupby('business_id')['name'].nunique().sum() == df['business_id'].nunique())

**Are there business sharing a name and address? Could they be the same business with multiple entries?**

In [ ]:
duplicates_per_id = df.groupby(['name', 'address'])['business_id'].nunique().reset_index()
duplicates_per_id = duplicates_per_id[duplicates_per_id['business_id'] > 1]

# Display the duplicates per business_id
print("Businesses with different IDs but the same name and address:")
print(duplicates_per_id)

**Seems like we do have multiple entries for a few businesses.**

**Keep only 1 business ID for each**

In [ ]:
df['business_id'] = df.groupby(['name', 'address'])['business_id'].transform('min')

**Re-calculate the review count for each business, as we mereged 2 data sets**

In [ ]:
#recalculate overall review count for each business id, as we merged 2 data sets
df['review_count'] = df.groupby('business_id')['customer_stars'].transform('count')

**Are there any business sharing a name?**

In [ ]:
print('unique names:', df['name'].nunique())
print('unique ids:', df['business_id'].nunique())

**Yes, there are. Let's try to figure out why:**

In [ ]:
df.groupby('name')['business_id'].nunique().sort_values(ascending=False).head(10)

**That makes sense - we have chains in the dataset. Nothing needs to be done about it**

**As seen before we have establishmets with different busines id sharing a name (most likely all chains with a number of branches). 
So, we'll create a unique display name for each business. 
If it's the only one by that name - just leave the name. If there are multiple businesses sharing the same name - add the state. If there are multiple sharing the name and state - add the city. Finally, if there are ones sharing name, city and state - add the address.
As seen before - address is nan for some businesses. If so, number the businesses 1, 2 and so on**

In [ ]:
display_name = df[['business_id', 'name', 'address', 'city', 'state_']].drop_duplicates(subset=['business_id','name']).reset_index(drop=True)
display_name['business_display'] = display_name['name']

# for duplicated names, set to name + state
display_name.loc[display_name.duplicated(subset=['name'], keep=False), 'business_display'] += ' - ' + display_name['state_']

# for duplicated name + state, set name + city + state
display_name.loc[display_name.duplicated(subset=['name', 'state_'], keep=False), 'business_display'] = display_name['name'] + ' - ' + display_name['city'] + ',' + display_name['state_']

# for duplicated name + city + state, set name + address + city + state
mask_duplicate_city_state = display_name.duplicated(subset=['name', 'city', 'state_'], keep=False)
display_name.loc[mask_duplicate_city_state, 'business_display'] = (
    display_name['name'] + ' - ' +
    display_name['address'].fillna('') +  # if address is nan, fill with empty str for now
    display_name['city'] + ',' + display_name['state_']
)

# for rows with nan address, add a unique number identifier at the end
display_name.loc[mask_duplicate_city_state & display_name['address'].isna(), 'business_display'] +=  (display_name[mask_duplicate_city_state].groupby(['name', 'city', 'state_']).cumcount() + 1).astype(str)
display_name.drop(['name', 'address', 'city', 'state_'], inplace=True, axis=1)

**Drop coloums that aren't useful to the analysis**

Note: Some coloumns, like categories, could be very interesting to make analysis on. But for the sake if this analysis and due to time constraints, it has not been used, so we drop it.


In [ ]:
df.drop(['postal_code', 'latitude', 'longitude', 'useful', 'funny', 'cool', 'hours', 'is_open', 'categories', 'user_id', 'review_id', 'name', 'address', 'city', 'state_'], inplace=True, axis=1)

**How many businesses do we have in the dataframe?**

In [ ]:
print('Number of businesses:', df['business_id'].nunique())

## Filter the data

**Filter the data to ensure we're working with restaurents with sufficient user reviews data.**

**Do so by the following criteria:**

**1 The restaurent has at least 200 user reviews in total**

**2 Theres a consecuative 60 days period in which the restaurent has at least 40 reviews**

In [ ]:
TOTAL_REVIEW_COUNT_CUTOFF = 200 # min reviews the restuarnt has to have to be included in the analysis
FILTER_WINDOW =  60 # rolling window used to filter out restaurents with insufficent data
REVIEW_COUNT_WINDOW_CUTOOF = 40 # used toghther with the rolling window to filter out restaurents

filtered_df = df[df['review_count'] >= TOTAL_REVIEW_COUNT_CUTOFF].copy() # filter for restaurents with at least 200 reviews
filtered_df['reviews_per_date'] = 1         # preparation for calculating the total reviews per date and eventually for each 60d period
                                            # we'll sum this column accross identical dates to calc the number of reviews for each date

condition = (
    filtered_df.set_index('date_')      # set the index to date, so we can resample by day later
    .groupby('business_id')             # ensure we're sampling the data by day *for each* business_id
    .resample('1D')['reviews_per_date'] # resample the data by day
    .sum()                              # sum the reviews_per_date column, to calculate the total number of reviews per each date
    .rolling(window=FILTER_WINDOW, min_periods=1) 
    .sum()                              # calculate the total reviews for each 60 day period
    .ge(REVIEW_COUNT_WINDOW_CUTOOF)    # are there at least 40 reviews?
    .groupby('business_id')             
    .any()                              # for each business_id, is there at least 1 period with more than 40 reviews?
)

filtered_df = filtered_df.set_index('business_id').loc[condition.index[condition]].reset_index()

**How many businesses are we left with?**

In [ ]:
print('Number of businesses:', filtered_df['business_id'].nunique())

## Identifying outlier periods

**Calculate the rolling rating mean and review count for each business using 3-week, year windows**

In [ ]:
# calculate the rolling rating mean and review count using the specified window
def calc_rolling_stats(date_sample, window):    
    rolling_result = date_sample.rolling(window = window).sum()
    rolling_result['rating_mean'] = rolling_result['customer_stars'] / rolling_result['reviews_per_date']
    return rolling_result.reset_index()

# like beforehand, we'll use this column to calculate the 
# total reviews per date and eventually the total reviews in each rolling window
# we'll sum this column accross identical dates to achieve the first point
filtered_df['reviews_per_date'] = 1 
all_dates = (
    filtered_df.set_index('date_')      # set the index to date, so we can resample by day later
    .groupby('business_id')             # ensure we're sampling the data by day *for each* business_id
    .resample('1D')[['customer_stars','reviews_per_date']] # resample the data by day
    .sum()                              # sum the reviews_per_date column, to calculate the total number of reviews per each date
)

# calculate the rolling mean and count results using 2 windows: 3 weeks, a year
rolling_result_3_weeks = all_dates.groupby('business_id', group_keys=False).apply(lambda business : calc_rolling_stats(business, DAYS_3_WEEKS)).reset_index(drop=True)
rolling_result_6_months = all_dates.groupby('business_id', group_keys=False).apply(lambda business : calc_rolling_stats(business, DAYS_6_MONTHS)).reset_index(drop=True)
all_dates.reset_index(inplace=True)


# sum it all up in one dataframe
rolling_stats = pd.concat(
     [all_dates,
     rolling_result_3_weeks['rating_mean'].rename('3_weeks'),
     rolling_result_3_weeks['reviews_per_date'].rename('count_3_weeks'),
     rolling_result_6_months['rating_mean'].rename('6_months'),
     rolling_result_6_months['reviews_per_date'].rename('count_6_months')],
     axis=1
)

**Filter for periods in which the average was significantly above or below the 6 month average**

Note: we only calculate it on periods which have enough data - at least 30 reviews in a year and at least 10 in a week. Otherwise, we'll get meaningless statistics

In [ ]:
ROLLING_6_MONTHS_CUTOFF = 30 # min reviews in 60 months to be considered in outlier period detection
ROLLING_3_WEEKS_CUTOFF = 10 # min reviews in 3 weeks to be considered in outlier period detection
STD_OUTLIRE_CUTOFF = 2 # number of standard deviation the rating has to be away from the 6-month mean
                       # to be considered an outlier
MIN_OUTLIER_CUTOFF = 0.45 # minimum deviation from the 6 months to be considered an outlier

# filter the rolling stats datafrmae for periods which have sufficient data
rolling_filtered = rolling_stats.query('count_6_months >= @ROLLING_6_MONTHS_CUTOFF & count_3_weeks >= @ROLLING_3_WEEKS_CUTOFF')[['business_id', 'date_', '6_months', '3_weeks', 'count_3_weeks', 'count_6_months']].copy()

# for each date (row), calculate the diffrence between the 6-month mean rating and the 3 weeks mean rating
rolling_filtered['diff_6_months_3_weeks'] = rolling_filtered['6_months'] - rolling_filtered['3_weeks']

rolling_filtered['diff_std'] = rolling_filtered.groupby('business_id')['diff_6_months_3_weeks'].transform('std')
rolling_filtered['diff_mean'] = rolling_filtered.groupby('business_id')['diff_6_months_3_weeks'].transform('mean')
rolling_filtered
# for each business, filter for outlier dates, defined by the year-3 weeks mean diff for the date
# being more or less the 2.5 * the std of the diff away from the mean of the diff
rolling_filtered = rolling_filtered[(np.abs(rolling_filtered['diff_6_months_3_weeks']) >= MIN_OUTLIER_CUTOFF) &
                                    ((rolling_filtered['diff_6_months_3_weeks'] < rolling_filtered['diff_mean'] - STD_OUTLIRE_CUTOFF * rolling_filtered['diff_std']) | 
                                     (rolling_filtered['diff_6_months_3_weeks'] > rolling_filtered['diff_mean'] + STD_OUTLIRE_CUTOFF * rolling_filtered['diff_std']))]

**As we might get a, outlier period starting on the 01-03, and one on the 01-04, and the next day and so on... (as expected), group overlapping outlier periods into 1, so the output would be easier to understand**

In [ ]:
# we need to sort the dataframe for the upcoming operations
rolling_filtered.sort_values(['business_id','date_'], inplace=True)

# add a new coloumn, equal 1 if it's consecutive to the date (row) before, 0 otherwise
rolling_filtered['consecutive'] = (('0d' < rolling_filtered['date_'].diff()) & (rolling_filtered['date_'].diff() < pd.Timedelta(days=DAYS_3_WEEKS))).astype(int).fillna('0d')

# create a cumulative sum to identify consecutive date groups
rolling_filtered['group'] = (rolling_filtered['consecutive'] == 0).cumsum()

# use the coloum created to figure out start and end dates of the consecutive period
rolling_filtered['start_date'] = pd.to_datetime(rolling_filtered.groupby('group')['date_'].transform('min'))
rolling_filtered['end_date'] = pd.to_datetime(rolling_filtered.groupby('group')['date_'].transform('max'))

# shift the start date by 20 days, as we calculate a 3 weeks period
rolling_filtered['start_date'] = rolling_filtered['start_date'] - pd.Timedelta(days=DAYS_3_WEEKS - 1)

# drop the helper temp coloumns
rolling_filtered.drop(['consecutive', 'group'], axis=1, inplace=True)

# create a copy of containing only 1 row for each pair of start_date and end_date, will use later
periods = rolling_filtered.drop_duplicates(subset=['business_id', 'start_date', 'end_date'])[['business_id', 'start_date', 'end_date','count_3_weeks', 'diff_6_months_3_weeks']].copy()


**Calculate the rating mean in each outlier period determined in the last operation**

code is quite robust - but its for efficiency purposes, as were dealing with a large data.
Direct boolean indexing takes a long, long time.

In [ ]:
# all_dates contains: for each business id, consecutive dates from the date of the first review 
# to the last.
# for each date, it contains the sum of total stars that users have given the restaurent
# (could be more than 5) and the total number of reviews for the date.
# this code finds the indices in all_dates that corrospond to the date
# of the first and last reviews in an outlier period for each business and use it to calculate the rating mean
# basically: for a given business id and a period: start date in all dates = index of the first row 
# in all_dates with the given business id + start date of the period - 
# date of the first review of the business

# we need to sort the dataframe for the upcoming operations
all_dates.sort_values(['business_id', 'date_'], inplace=True)
periods['period_length'] = (periods['end_date'] - periods['start_date']).dt.days

# create a dict containing the date the first review was written on each business
# and the index in all_dates if the first review for each business
min_dates_dict = all_dates.reset_index().groupby('business_id').agg({'date_': 'min', 'index': 'first'}).to_dict()

# the date the first review was written on for each business
periods['min_date'] = periods['business_id'].map(min_dates_dict['date_'])

# the index in all_dates for the first review of the business
periods['first_index'] = periods['business_id'].map(min_dates_dict['index'])

# calc the index of the date of the first review for each business in the df all_dates
periods['index_in_all_dates'] = (periods['start_date'] - periods['min_date']).dt.days + periods['first_index']

# iterate over the outlier periods
for idx, row in periods.iterrows():
    # the first index of the date in the outlier period, in all_dates
    start_index = row['index_in_all_dates']
    # the index of the last date
    end_index = start_index + int(row['period_length'])
    # calc total reviews
    total_reviews = all_dates.loc[start_index:end_index, 'reviews_per_date'].sum()
    if total_reviews > 0:
        # calc the rating mean
        mean_value = (all_dates.loc[start_index:end_index, 'customer_stars']).sum() / total_reviews
    else:
        mean_value = 0
    
    # update the df with the rating
    periods.at[idx, 'period_mean_rating'] = mean_value

# drop temp helper coloumns
periods.drop(['min_date', 'first_index', 'index_in_all_dates'], axis=1, inplace=True)


## Visualizing the results

Note: due to the filtering before, only periods which have at least 6 reviews and at least 30 in the 6 months before them will count as outliers. A period with less than 10 reviews dosen't have sufficient data to be classified as an outlire. This is why some periods which seem abnormaly low or high aren't markesd as outliers (in red) in the graph. You may notice that the size of these dots is relatively small, indicating that in that time period there were indeed just a very few reviews.

In [ ]:
# on business selection from the dropdown
def on_business_selection(business_display):
    business_id = display_name[display_name['business_display'] == business_display]['business_id'].iloc[0]
    update_graph(business_id)
    update_periods_dropdown(business_id)

# update the outlier periods graph. it contains plots the 6 months moving average, 3 week moving average
# and marks the oulier periods in red
def update_graph(business_id):
    # get the raw outlier data for the business id - 
    # not the merged period we calculated in the 'periods' df
    business_outlires_data = rolling_filtered[rolling_filtered['business_id'] == business_id]
    # get the entire business data
    business_data = rolling_stats[rolling_stats['business_id'] == business_id]
    
    # create the graph figure, add labels and title
    fig = go.Figure()
    fig.update_layout(
        title_text ='Outlier 3 Week Periods Based on 3-Week Average Rating Compared to Average 6-Month Rating',
        xaxis_title = 'Date',
        yaxis_title = 'Average Rating' 
    )
    
    # plot the 3 week moving average - ignore 0 values where there were no reviews
    positive = business_data[business_data['3_weeks'] > 0]
    hover_3_weeks = '<b>Period:</b> '+ (positive['date_'] - pd.DateOffset(20)).dt.date.astype(str) + ' - ' + positive['date_'].dt.date.astype(str) + '<br><b>Rating:</b>' + positive['3_weeks'].astype(str) + '<br><b>Review Count:</b> ' + positive['count_3_weeks'].astype(str)    
    avg_3_weeks = go.Scatter(
        x=positive['date_'], 
        y=positive['3_weeks'], 
        mode='markers', 
        marker=dict(size=positive['count_3_weeks'], color='blue'), 
        name='moving 3 weeks rating average<br>sized by review count',
        text = hover_3_weeks
    )
    fig.add_trace(avg_3_weeks)
    
    # plot the 3 week moving average outlier periods
    outliers_hover = '<b>Period:</b> '+ business_outlires_data['start_date'].dt.date.astype(str) + ' - ' + business_outlires_data['end_date'].dt.date.astype(str) + '<br><b>Rating:</b>' + business_outlires_data['3_weeks'].astype(str) + '<br><b>Review Count:</b> ' + business_outlires_data['count_3_weeks'].astype(str)
    outliers = go.Scatter(
        x=business_outlires_data['date_'], 
        y=business_outlires_data['3_weeks'], 
        mode='markers', 
        marker=dict(size=business_outlires_data['count_3_weeks'], 
        color='red', line=dict(width=0)), 
        name='outlier 3 week periods',
        text=outliers_hover
    )
    fig.add_trace(outliers)
    
    # plot the 6 months moving average
    hover_6_months = '<b>Date:</b>' + business_data['date_'].dt.date.astype(str) + '<br><b>Rating:<b>' + business_data['6_months'].astype(str)
    avg_6_months = go.Scatter(
        x=business_data['date_'], 
        y=business_data['6_months'], 
        mode='lines', 
        line_shape='linear', 
        line=dict(color='orange', width=5), 
        name='moving 6-months rating average',
        text = hover_6_months
    )
    fig.add_trace(avg_6_months)

    fig.show()

# update the outlier periods selection dropdown
def update_periods_dropdown(business_id):
    outliers = periods[periods['business_id'] == business_id]
    period_options = outliers.apply(lambda row: f"{row['start_date']} - {row['end_date']}", axis=1)
    period_dropdown.options = period_options.unique()

# show the reviews from the outlier period that have a rating that is below/above the mean of the whole
# period, according to if it falls below/above the 6-month mean
def show_reviews(business_id, period):
    start_date = pd.to_datetime(period[:10])
    end_date = pd.to_datetime(period[22:]) + pd.DateOffset(1)
    
    business_period_data = filtered_df[(filtered_df['business_id'] == business_id) & (filtered_df['date_'] >= start_date) & (filtered_df['date_'] <= end_date)]
    outlier_period_data = periods[(periods['business_id'] == business_id) & (periods['start_date'] == start_date)].iloc[0]
    
    # is the mean of the period higher than that of the 6 months before
    is_positive = outlier_period_data['diff_6_months_3_weeks'] < 0
    
    # we'll use this to filter relevant reviews
    cutoff = outlier_period_data['period_mean_rating']
    
    # take the reviews that are above the mean if the period has a mean above that of the 6 months prior,
    # otherwise take those that are below the mean
    data  = business_period_data[(is_positive and business_period_data['customer_stars'] >= cutoff) | (not is_positive and business_period_data['customer_stars'] <= cutoff)]
    
    
    # show the reviews in a markdown
    with output:
        clear_output(wait=True)
        display(Markdown('### Reviews From the Selected Period:<br>'))
        for _, row in data.sort_values('date_').iterrows():
            display(Markdown(str(row['date_']) + ' rating: ' + str(row['customer_stars']) + '<br>' + row['text_']))


# show bar charts comparing the rating and rating ditriutions across 3 periods:
# the selected outlier period, the 3 weeks prior, 6 months prior
def show_rating_bar_charts(period, business_id):
    business_data = filtered_df[filtered_df['business_id'] == business_id]
    start_date = pd.to_datetime(period[:10])
    end_date = pd.to_datetime(period[22:]) + pd.DateOffset(1)
    
    def get_period_counts(start, end):
        # get all the reviews from the time period
        period_data = business_data[(business_data['date_'] >= start) & (business_data['date_'] <= end)]
        # get total count of reviews in the period
        total_ratings = period_data['customer_stars'].count()
        # get the precentage of each rating 1-5 in the time period
        return (period_data['customer_stars'].value_counts().sort_index() / total_ratings) * 100
    
    # Get ratings distribution for each period
    period_dist = get_period_counts(start_date, end_date)
    prev_3_weeks_dist = get_period_counts(start_date - pd.DateOffset(DAYS_3_WEEKS), start_date)
    prev_6_months_dist = get_period_counts(end_date - pd.DateOffset(DAYS_6_MONTHS), end_date)
    
    # for plotting the distribution
    plot_data = pd.DataFrame({
        'Previous 3 Weeks': prev_3_weeks_dist,
        'Outlier Period': period_dist,
        'Previous 6 Months': prev_6_months_dist
    })
    print(plot_data)
    # plot the ditribution
    fig_rating_distribution = px.bar(plot_data, x=plot_data.index.to_list(), y=plot_data.columns,
                 labels={'index': 'Rating', 'value': 'Percentage'}, 
                 title='Rating Distribution (Percentage) Over Different Periods',
                 height=400,
                 barmode='group')
    
    # set x and y axis labels
    fig_rating_distribution.update_layout(
        xaxis_title='Rating',
        yaxis_title='Percentage'
    )
    
    # get the rolling stats for the business id
    rolling_business = rolling_stats[rolling_stats['business_id'] == business_id]
    # get the mean for the prev 6 months
    mean_6_months = rolling_business[rolling_business['date_'] == end_date]['6_months'].iloc[0]
    # get the mean for the prev 3 weeks
    rating_prev_3_weeks = rolling_business[rolling_business['date_'] == start_date - pd.DateOffset(DAYS_3_WEEKS)]['3_weeks'].iloc[0]
    # get the mean for the selected outlier
    rating_outlier = periods[(periods['business_id'] == business_id) & (periods['start_date'] == start_date)]['period_mean_rating'].iloc[0]
    
    # plot the mean across the 3 periods
    fig_rating_accross_periods = px.bar(x=['Previous 3 Weeks', 'Outlier Period', 'Previous 6 Months'], 
                  y = [rating_prev_3_weeks, rating_outlier,mean_6_months], 
                  color=['Previous 3 Weeks', 'Outlier Period', 'Previous 6 Months'],
                  title = 'Mean Rating Over Different Periods',
                  height = 400)
    
    # aet x and y axis labels
    fig_rating_accross_periods.update_layout(
        xaxis_title='Period',
        yaxis_title='Rating'
    )
    
    # show the graphs
    fig_rating_accross_periods.show()
    fig_rating_distribution.show()

# updates on period selection change
def period_selection_update(period):
    business_display = business_dropdown.value
    business_id = display_name[display_name['business_display'] == business_display]['business_id'].iloc[0]
    show_rating_bar_charts(period, business_id)
    show_reviews(business_id, period)
    
business_with_outliers = periods['business_id'].unique()
business_displays = display_name.loc[display_name['business_id'].isin(business_with_outliers), 'business_display'].sort_values().tolist()

business_dropdown = widgets.Dropdown(
    options = business_displays,
    description='Select Business'
)


# create a outlire period dropdown list
period_dropdown = widgets.Dropdown(description='Period:')
output = widgets.Output()

# create interactive plots
business_interactive_plot = widgets.interactive(on_business_selection, business_display = business_dropdown)
period_interactive_plot = widgets.interactive(period_selection_update, period = period_dropdown)

# Display the business dropdown, period dropdown, and output
display(business_interactive_plot, period_interactive_plot, output)

## Let's take a look at an example - Old Lady Gang - 177 Peters St SW,  Atlanta,  GA

In [ ]:
update_graph('QW3anlt1nJ9aNkdQAV-Ifw')

**Let's take a closer look on the period: 2019-02-15 - 2019-03-27**

In [ ]:
show_rating_bar_charts('2019-02-15 00:00:00 - 2019-03-27 00:00:00', 'QW3anlt1nJ9aNkdQAV-Ifw')

**We see that the rating and rating distribution for this time period was indeed abnormal, compared to the 6 months before and the 3 weeks before.**
**The overall rating falls off ~0.8 from the rating of the 6 months before, and more than 40% of the reviews were star rating of 1, compared to 10% or less in the other 2 periods.**
**Let's take a look at a summery of the negative reviews themselves:**

**Negative Dining Experience:**

"The hostess was the absolute worst... turned away 5 groups of people because they didn't have a reservation."
"She rolls her eyes then seats us at the table closest to the entrance... not worth having to deal with someone with incredibly poor customer service." (2019-02-15)

**Reservation Issues and Poor Communication:**

"Flew all the way from Florida, made a reservation for my birthday... received a cancellation email... location was closed for reservations." (2019-02-23)
"They could have included that in the cancellation email so I didn't have to waste a $50 Uber ride. Poor customer service." (2019-02-23)

"Upsetting that this location was closed for renovation and the website did not list that... over an hour drive from our hotel just to find it closed." (2019-02-25)

"Asked workers why isn't this information on your website?... I'm more so pissed for dragging my elderly, handicapped parents here just to have my dad huff and puff back to the car."(2019-02-28)

"I made a reservation... arrived...only to find a piece of paper on the window stating that this location was closed for renovations!" (2019-03-18)

**Challenges with Wait Times and Alleged Discrimination:**

"Wait time was 1 hour and 45 minutes... after waiting 45 minutes, told the wait time was still 1 1/2 hours... Candie and Tod, instead of trying to open more, run it like an establishment that I would like to recommend." (2019-03-04)

"Black people think because their customers are black that they can treat them any kind of way... teach the host and hostess how to tell time so they can seat guests properly." (2019-03-04)



**We see that the main issue in this time period, which affected greatly on the reviews and customer satisfication, was that the place was closed for renovations - but didn't advertise it correctly (4 reviews mentioned that, and we can assume many more people were affected by this but didn't leave a review). Thats a point the business should take into considiration if they have to close the location temporarily again.**

**Now let's take a look at a positive period: 2019-11-10 - 2019-12-28:**

In [ ]:
show_rating_bar_charts('2019-11-10 00:00:00 - 2019-12-28 00:00:00', 'QW3anlt1nJ9aNkdQAV-Ifw')

**We see that the rating in this time period is almost a full point above that of the 6 months before.**

**Let's take a look at the summery of the positive reviews:**

**Exceptional Food Quality and Diverse Menu Options**

"My wife and I ate lunch there today and the food as well as the service was great!... We're going back!" (Nov 12, 2019)

"This was my first time visiting this restaurant... They did not disappoint!... Our food was perfect! I had the fried chicken entree with a shrimp and grits appetizer. AMAZING." (Nov 12, 2019)

"Great cozy spot... Food was great." (Nov 13, 2019)

"Enjoyed the food and drinks..." (Nov 17, 2019) 

"Unfortunately, the deep-fried deviled eggs were all sold out... Salmon bites were a HIT!" (Nov 19, 2019) 

"The Salmon Bites were good..." (Nov 26, 2019)

"I'm sooo glad I ignored the negative comments and tried for myself... EVERYTHING was tasty!!... We ordered the fried deviled eggs w/ grilled shrimp." (Nov 29, 2019)

"This is definitely a 5-star restaurant... The food is excellent... I had the lamb chops and smashed potatoes. The lamb chops were tender and cooked to perfection." (Nov 30, 2019)

"The food was great... We had the fried eggs and fried salmon delicious!!!!" (Dec 8, 2019)

"I throughly enjoyed the food!... 5-star all the way for the food!!" (Dec 14, 2019)

"As expected, the food and drink pours were on point!! I ordered the blackened salmon with mashed potatoes and green beans. Everything was cooked to perfection." (Dec 23, 2019)

"I've been wanting to try this place for a while... I ordered the wings and fried deviled eggs. The eggs were amazing!" (Dec 27, 2019)

"The fried chicken was seasoned to perfection!... I recommend the fried chicken, greens, and Mac and cheese." (Dec 27, 2019)

**Outstanding Service and Hospitality**

"Our waitress Kera was so sweet!... Have you ever met someone that makes you involuntarily smile from the level of happiness within the convo? No? Go meet Kera!" (Nov 12, 2019)

"The host, Corrina was absolutely a breath of fresh air! Her level of professionalism while still being extremely kind and pleasant was premier! Our waitress Kera was so sweet!" (Nov 12, 2019)

"Waitress was nice and attentive." (Nov 13, 2019)

"Bartender Do was awesome!!! Went on with no reservations and sat at the bar, got treated with love." (Nov 17, 2019)

"Nice, bougie atmosphere... Our server was super friendly and very helpful while ordering." (Nov 19, 2019)

"And my waitress was beyond pleasant... Love the option of sitting upstairs or downstairs." (Nov 26, 2019)

"Our waitress was perfect and able to answer my menu inquiries... No long waits, excellent service, great food!" (Nov 29, 2019)

"Aunt Nora was here and she was so kind and hospitable..." (Nov 30, 2019)

"The bartender was great Teria very personable great customer service." (Nov 30, 2019)

"The music played was amazing... I was very impressed how the owner interacted with guests... I would definitely go back." (Nov 30, 2019)

"Both bartenders were nice, professional, and courteous... Dominique pretty much took care of me... I was totally impressed with OLG!" (Dec 2, 2019)

"Julie was a joy to meet and was extremely attentive. The atmosphere is homey, and casual, pictures of Kandi and her family all over the walls... This was a simply amazing experience and would recommend this to anyone and everyone!!" (Dec 11, 2019)

"Our waitress was super nice and attentive... I will definitely be back." (Dec 9, 2019)
"Dee my server was great... The food is always good and the service awesome." (Dec 9, 2019)
"Our drinks were so delicious and she definitely is one of the most unique and talented bartenders I have ever met... We will be back soon." (Dec 9, 2019)

"Julie was awesome and so was Corina... We will be back again." (Dec 11, 2019)

"The staff is very friendly..." (Dec 27, 2019)

"The customer service was superb!" (Dec 27, 2019)

**We see that in this time period the food and service were exceptional. The reviews even mention staff by name. By continuing with doing what the restaurent did in this period, it could definitley start an upward trend in customer satification and ratings.**

## Conclusion

**by examaning the user reviews over time, we could find outlier periods that each restaurent could take into considaration if it wants to improve on its' performance.**

**What I expected to see is that a negative outlier period will often corrolate with a negative long-term rating trend, and a positive one will often corrolates with a long-term positive trend.**

**If so,  we could recommend the businesses to track the outlier periods in real time, in hopes of preventing long term negative trends and maintaining long term potive trends.**

**However, its not really clear if that's the case. Either way, tracking outlier period can still provide businesses with valubale information on its performance as measured by user reviews.**